In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [39]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [40]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

In [41]:
import sklearn
from sklearn.metrics import classification_report, confusion_matrix

## WARNING, FILE "delay_clean.csv" is > 4 GB  --  added to gitignore
## Using reduced dataset.....

# Gradient-boosted tree classifier (GBT)

In [42]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [43]:
# Load and parse the data file, converting it to a DataFrame
clean = spark.read.format("libsvm").load('/content/drive/MyDrive/Colab_Notebooks/delay_clean2K_SVM.txt')
# clean = spark.read.csv('/content/drive/MyDrive/Colab_Notebooks/delay_clean2K.csv', header = True, inferSchema = True)

In [44]:
# FOR VECTOR ASSEMBLER
clean.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(137,[0,1,2,3,4,5...|
|  0.0|(137,[0,1,2,3,4,5...|
|  0.0|(137,[0,1,2,3,4,5...|
|  0.0|(137,[0,1,2,3,4,5...|
|  0.0|(137,[0,1,2,3,4,5...|
+-----+--------------------+
only showing top 5 rows



In [9]:
#from pyspark.ml.feature import VectorAssembler

In [ ]:
#fullColList = clean.columns
#fullColList

## COPY list from above and REMOVE manually DEP_DEL_15 from the LIST

In [11]:
inputColList = ['MONTH',
 'DAY_OF_WEEK',
 'DISTANCE_GROUP',
 'SEGMENT_NUMBER',
 'CONCURRENT_FLIGHTS',
 'NUMBER_OF_SEATS',
 'AIRPORT_FLIGHTS_MONTH',
 'AIRLINE_FLIGHTS_MONTH',
 'AIRLINE_AIRPORT_FLIGHTS_MONTH',
 'AVG_MONTHLY_PASS_AIRPORT',
 'AVG_MONTHLY_PASS_AIRLINE',
 'FLT_ATTENDANTS_PER_PASS',
 'GROUND_SERV_PER_PASS',
 'PLANE_AGE',
 'PREVIOUS_AIRPORT',
 'PRCP',
 'SNOW',
 'SNWD',
 'TMAX',
 'AWND',
 'DEP_TIME_BLK_0001_0559',
 'DEP_TIME_BLK_0600_0659',
 'DEP_TIME_BLK_0700_0759',
 'DEP_TIME_BLK_0800_0859',
 'DEP_TIME_BLK_0900_0959',
 'DEP_TIME_BLK_1000_1059',
 'DEP_TIME_BLK_1100_1159',
 'DEP_TIME_BLK_1200_1259',
 'DEP_TIME_BLK_1300_1359',
 'DEP_TIME_BLK_1400_1459',
 'DEP_TIME_BLK_1500_1559',
 'DEP_TIME_BLK_1600_1659',
 'DEP_TIME_BLK_1700_1759',
 'DEP_TIME_BLK_1800_1859',
 'DEP_TIME_BLK_1900_1959',
 'DEP_TIME_BLK_2000_2059',
 'DEP_TIME_BLK_2100_2159',
 'DEP_TIME_BLK_2200_2259',
 'DEP_TIME_BLK_2300_2359',
 'CARRIER_NAME_Alaska_Airlines_Inc',
 'CARRIER_NAME_Allegiant_Air',
 'CARRIER_NAME_American_Airlines_Inc',
 'CARRIER_NAME_American_Eagle_Airlines_Inc',
 'CARRIER_NAME_Atlantic_Southeast_Airlines',
 'CARRIER_NAME_Comair_Inc',
 'CARRIER_NAME_Delta_Air_Lines_Inc',
 'CARRIER_NAME_Endeavor_Air_Inc',
 'CARRIER_NAME_Frontier_Airlines_Inc',
 'CARRIER_NAME_Hawaiian_Airlines_Inc',
 'CARRIER_NAME_JetBlue_Airways',
 'CARRIER_NAME_Mesa_Airlines_Inc',
 'CARRIER_NAME_Midwest_Airline,_Inc',
 'CARRIER_NAME_SkyWest_Airlines_Inc',
 'CARRIER_NAME_Southwest_Airlines_Co',
 'CARRIER_NAME_Spirit_Air_Lines',
 'CARRIER_NAME_United_Air_Lines_Inc',
 'DEPARTING_AIRPORT_Albuquerque_International_Sunport',
 'DEPARTING_AIRPORT_Anchorage_International',
 'DEPARTING_AIRPORT_Atlanta_Municipal',
 'DEPARTING_AIRPORT_Austin___Bergstrom_International',
 'DEPARTING_AIRPORT_Birmingham_Airport',
 'DEPARTING_AIRPORT_Boise_Air_Terminal',
 'DEPARTING_AIRPORT_Bradley_International',
 'DEPARTING_AIRPORT_Charleston_International',
 'DEPARTING_AIRPORT_Chicago_Midway_International',
 'DEPARTING_AIRPORT_Chicago_OHare_International',
 'DEPARTING_AIRPORT_Cincinnati_Northern_Kentucky_International',
 'DEPARTING_AIRPORT_Cleveland_Hopkins_International',
 'DEPARTING_AIRPORT_Dallas_Fort_Worth_Regional',
 'DEPARTING_AIRPORT_Dallas_Love_Field',
 'DEPARTING_AIRPORT_Des_Moines_Municipal',
 'DEPARTING_AIRPORT_Detroit_Metro_Wayne_County',
 'DEPARTING_AIRPORT_Douglas_Municipal',
 'DEPARTING_AIRPORT_El_Paso_International',
 'DEPARTING_AIRPORT_Eppley_Airfield',
 'DEPARTING_AIRPORT_Fort_Lauderdale_Hollywood_International',
 'DEPARTING_AIRPORT_Friendship_International',
 'DEPARTING_AIRPORT_General_Mitchell_Field',
 'DEPARTING_AIRPORT_Greater_Buffalo_International',
 'DEPARTING_AIRPORT_Greenville_Spartanburg',
 'DEPARTING_AIRPORT_Hollywood_Burbank_Midpoint',
 'DEPARTING_AIRPORT_Honolulu_International',
 'DEPARTING_AIRPORT_Houston_Intercontinental',
 'DEPARTING_AIRPORT_Indianapolis_Muni_Weir_Cook',
 'DEPARTING_AIRPORT_Jacksonville_International',
 'DEPARTING_AIRPORT_John_F_Kennedy_International',
 'DEPARTING_AIRPORT_Kansas_City_International',
 'DEPARTING_AIRPORT_Keahole',
 'DEPARTING_AIRPORT_Kent_County',
 'DEPARTING_AIRPORT_LaGuardia',
 'DEPARTING_AIRPORT_Lambert_St_Louis_International',
 'DEPARTING_AIRPORT_Logan_International',
 'DEPARTING_AIRPORT_Los_Angeles_International',
 'DEPARTING_AIRPORT_Louis_Armstrong_New_Orleans_International',
 'DEPARTING_AIRPORT_McCarran_International',
 'DEPARTING_AIRPORT_McGhee_Tyson',
 'DEPARTING_AIRPORT_Memphis_International',
 'DEPARTING_AIRPORT_Metropolitan_Oakland_International',
 'DEPARTING_AIRPORT_Miami_International',
 'DEPARTING_AIRPORT_Minneapolis_St_Paul_International',
 'DEPARTING_AIRPORT_Nashville_International',
 'DEPARTING_AIRPORT_Newark_Liberty_International',
 'DEPARTING_AIRPORT_Norfolk_International',
 'DEPARTING_AIRPORT_Ontario_International',
 'DEPARTING_AIRPORT_Orange_County',
 'DEPARTING_AIRPORT_Orlando_International',
 'DEPARTING_AIRPORT_Palm_Beach_International',
 'DEPARTING_AIRPORT_Palm_Springs_International',
 'DEPARTING_AIRPORT_Philadelphia_International',
 'DEPARTING_AIRPORT_Phoenix_Sky_Harbor_International',
 'DEPARTING_AIRPORT_Piedmont_Triad_International',
 'DEPARTING_AIRPORT_Pittsburgh_International',
 'DEPARTING_AIRPORT_Port_Columbus_International',
 'DEPARTING_AIRPORT_Portland_International',
 'DEPARTING_AIRPORT_Raleigh_Durham_International',
 'DEPARTING_AIRPORT_Reno_Tahoe_International',
 'DEPARTING_AIRPORT_Richmond_International',
 'DEPARTING_AIRPORT_Ronald_Reagan_Washington_National',
 'DEPARTING_AIRPORT_Sacramento_International',
 'DEPARTING_AIRPORT_Salt_Lake_City_International',
 'DEPARTING_AIRPORT_San_Antonio_International',
 'DEPARTING_AIRPORT_San_Diego_International_Lindbergh_Fl',
 'DEPARTING_AIRPORT_San_Francisco_International',
 'DEPARTING_AIRPORT_San_Jose_International',
 'DEPARTING_AIRPORT_Savannah_Hilton_Head_International',
 'DEPARTING_AIRPORT_Seattle_International',
 'DEPARTING_AIRPORT_Southwest_Florida_International',
 'DEPARTING_AIRPORT_Standiford_Field',
 'DEPARTING_AIRPORT_Stapleton_International',
 'DEPARTING_AIRPORT_Tampa_International',
 'DEPARTING_AIRPORT_Theodore_Francis_Green_State',
 'DEPARTING_AIRPORT_Tucson_International',
 'DEPARTING_AIRPORT_Tulsa_International',
 'DEPARTING_AIRPORT_Washington_Dulles_International',
 'DEPARTING_AIRPORT_Will_Rogers_World',
 'DEPARTING_AIRPORT_William_P_Hobby',
 'principal_component']

In [19]:
# Input all the features in one vector column
assembler = VectorAssembler(inputCols = inputColList, outputCol = 'features')

In [20]:
output = assembler.transform(clean)

In [21]:
output.show(5)

+---------+-----+-----------+--------------+--------------+------------------+---------------+---------------------+---------------------+-----------------------------+------------------------+------------------------+-----------------------+--------------------+---------+----------------+----+----+----+----+----+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+--------------------------------+--------------------------+----------------------------------+----------------------------------------+----------------------------------------+-----------------------+--------------------------------+--------------

In [22]:
# Display features and label column
output.select("DEP_DEL15", "features").show(5)

+---------+--------------------+
|DEP_DEL15|            features|
+---------+--------------------+
|        0|(137,[0,1,2,3,4,5...|
|        0|(137,[0,1,2,3,4,5...|
|        0|(137,[0,1,2,3,4,5...|
|        0|(137,[0,1,2,3,4,5...|
|        0|(137,[0,1,2,3,4,5...|
+---------+--------------------+
only showing top 5 rows



In [32]:
output.printSchema

<bound method DataFrame.printSchema of +---------+-----+-----------+--------------+--------------+------------------+---------------+---------------------+---------------------+-----------------------------+------------------------+------------------------+-----------------------+--------------------+---------+----------------+----+----+----+----+----+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+--------------------------------+--------------------------+----------------------------------+----------------------------------------+----------------------------------------+-----------------------+--------

In [24]:
output.printSchema()

root
 |-- DEP_DEL15: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- DISTANCE_GROUP: integer (nullable = true)
 |-- SEGMENT_NUMBER: integer (nullable = true)
 |-- CONCURRENT_FLIGHTS: integer (nullable = true)
 |-- NUMBER_OF_SEATS: integer (nullable = true)
 |-- AIRPORT_FLIGHTS_MONTH: integer (nullable = true)
 |-- AIRLINE_FLIGHTS_MONTH: integer (nullable = true)
 |-- AIRLINE_AIRPORT_FLIGHTS_MONTH: integer (nullable = true)
 |-- AVG_MONTHLY_PASS_AIRPORT: integer (nullable = true)
 |-- AVG_MONTHLY_PASS_AIRLINE: integer (nullable = true)
 |-- FLT_ATTENDANTS_PER_PASS: double (nullable = true)
 |-- GROUND_SERV_PER_PASS: double (nullable = true)
 |-- PLANE_AGE: integer (nullable = true)
 |-- PREVIOUS_AIRPORT: integer (nullable = true)
 |-- PRCP: double (nullable = true)
 |-- SNOW: double (nullable = true)
 |-- SNWD: double (nullable = true)
 |-- TMAX: double (nullable = true)
 |-- AWND: double (nullable = true)
 |-- DEP_TIME_BL

In [25]:
# Number of rows in dataset
row = output.count()
row

200000

In [26]:
# Index labels, adding metadata to the label column
# Fit on whole dataset to include all labels in index
labelIndexer = StringIndexer(inputCol = "DEP_DEL15", outputCol = "indexedLabel").fit(output)

In [27]:
# Automatically identify categorical features, and index them
# Set maxCategories so features with > 4 distinct values are treated as continuous
featureIndexer = VectorIndexer(inputCol = "features", outputCol = "indexedFeatures", maxCategories = 4).fit(output)

In [33]:
from pyspark.ml.feature import Normalizer

In [36]:
normalizer = Normalizer(inputCol = "features", outputCol = "normFeatures", p = 1.0)
NormOutput = normalizer.transform(clean)

IllegalArgumentException: ignored

In [28]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = output.randomSplit([0.7, 0.3])

In [29]:
trainingData.show(5)

+---------+-----+-----------+--------------+--------------+------------------+---------------+---------------------+---------------------+-----------------------------+------------------------+------------------------+-----------------------+--------------------+---------+----------------+----+----+----+----+-----+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+--------------------------------+--------------------------+----------------------------------+----------------------------------------+----------------------------------------+-----------------------+--------------------------------+-------------

In [30]:
testData.show(5)

+---------+-----+-----------+--------------+--------------+------------------+---------------+---------------------+---------------------+-----------------------------+------------------------+------------------------+-----------------------+--------------------+---------+----------------+----+----+----+----+-----+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+--------------------------------+--------------------------+----------------------------------+----------------------------------------+----------------------------------------+-----------------------+--------------------------------+-------------

In [ ]:
# Train a GBT model
gbt = GBTClassifier(labelCol = "indexedLabel", featuresCol = "indexedFeatures", maxIter = 10)

In [ ]:
# Chain indexers and GBT in a Pipeline
pipeline = Pipeline(stages = [labelIndexer, featureIndexer, gbt])

In [ ]:
# Train model.  This also runs the indexers
model = pipeline.fit(trainingData)

In [ ]:
# Make predictions
predictions = model.transform(testData)

In [ ]:
# Select example rows to display
predictions.select("prediction", "indexedLabel", "features").show(5)

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         0.0|(137,[0,1,2,3,4,5...|
|       0.0|         0.0|(137,[0,1,2,3,4,5...|
|       0.0|         0.0|(137,[0,1,2,3,4,5...|
|       0.0|         0.0|(137,[0,1,2,3,4,5...|
|       1.0|         0.0|(137,[0,1,2,3,4,5...|
+----------+------------+--------------------+
only showing top 5 rows



In [ ]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol = "indexedLabel", predictionCol = "prediction", metricName = "accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.82972
Test Error = 0.17028


In [ ]:
gbtModel = model.stages[2]
print(gbtModel)  # summary only

GBTClassificationModel: uid = GBTClassifier_65416a6e3fc4, numTrees=10, numClasses=2, numFeatures=137


In [ ]:
y_true = predictions.select(['indexedLabel']).collect()
y_pred = predictions.select(['prediction']).collect()

In [ ]:
print(confusion_matrix(y_true, y_pred)) # only TEST data is used (30% of 200,000 total rows)

[[49096   383]
 [ 9843   732]]


In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       0.83      0.99      0.91     49479
         1.0       0.66      0.07      0.13     10575

    accuracy                           0.83     60054
   macro avg       0.74      0.53      0.52     60054
weighted avg       0.80      0.83      0.77     60054



In [ ]:
gbt.save("gbt_model.model")

# Random forest classifier (RFC)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString

In [ ]:
# Train a RandomForest model
rf = RandomForestClassifier(labelCol = "indexedLabel", featuresCol = "indexedFeatures", numTrees = 10)

In [ ]:
# Convert indexed labels back to original labels
labelConverter = IndexToString(inputCol = "prediction", outputCol = "predictedLabel",
                               labels = labelIndexer.labels)

In [ ]:
# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages = [labelIndexer, featureIndexer, rf, labelConverter])

In [ ]:
# Train model.  This also runs the indexers
model = pipeline.fit(trainingData)

In [ ]:
# Make predictions
predictions1 = model.transform(testData)

In [ ]:
# Select example rows to display
predictions1.select("predictedLabel", "label", "features").show(5)

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|           0.0|  0.0|(137,[0,1,2,3,4,5...|
|           0.0|  0.0|(137,[0,1,2,3,4,5...|
|           0.0|  0.0|(137,[0,1,2,3,4,5...|
|           0.0|  0.0|(137,[0,1,2,3,4,5...|
|           0.0|  0.0|(137,[0,1,2,3,4,5...|
+--------------+-----+--------------------+
only showing top 5 rows



In [ ]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol = "indexedLabel", predictionCol = "prediction", metricName = "accuracy")
accuracy = evaluator.evaluate(predictions1)
print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.823654
Test Error = 0.176346


In [ ]:
rfModel = model.stages[2]
print(rfModel)  # summary only

RandomForestClassificationModel: uid=RandomForestClassifier_1051dcbb9c41, numTrees=10, numClasses=2, numFeatures=137


In [ ]:
print(rfModel.featureImportances)

(137,[1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,19,21,22,23,25,28,33,34,35,36,37,45,57,58,60,65,68,76,89,91,129,136],[0.002660787090102533,0.00035223639064922057,0.011034488412578926,0.005980531575021249,0.0268728501930391,0.01255701207683193,0.010112045273101467,0.03138039306205343,0.019519441265388255,0.0046578071960948425,0.009664596789815688,0.0007050394197125379,0.021592993005491706,0.13449445728240195,0.046969780470165284,0.11462445517291064,0.22751679179819742,0.00884471409553305,0.04890562480873259,0.05585033291354542,0.030341525875597124,0.0002879433319649791,0.00019617524172224518,0.011982875241493756,0.003763092501249444,0.003910330987504793,0.0020737133118537234,0.002172821710871158,0.031175939497662154,0.0011252762930813976,0.0005677588646960306,0.00036957110683012457,0.01731754032267286,0.001780723107836594,0.002362403298749672,0.06055340448848664,0.0013214765253683224,0.0012031417362913003,0.03319790826470023])


In [ ]:
y_true = predictions1.select(['indexedLabel']).collect()
y_pred = predictions1.select(['prediction']).collect()

In [ ]:
print(confusion_matrix(y_true, y_pred)) # only TEST data is used (30% of 200,000 total rows)

[[49626     0]
 [10625     0]]


In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       0.82      1.00      0.90     49626
         1.0       0.00      0.00      0.00     10625

    accuracy                           0.82     60251
   macro avg       0.41      0.50      0.45     60251
weighted avg       0.68      0.82      0.74     60251



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Factorization machines classifier

In [ ]:
from pyspark.ml.classification import FMClassifier
from pyspark.ml.feature import MinMaxScaler

In [ ]:
# Index labels, adding metadata to the label column
# Fit on whole dataset to include all labels in index
labelIndexer = StringIndexer(inputCol = "label", outputCol = "indexedLabel").fit(clean)

# Scale features
featureScaler = MinMaxScaler(inputCol = "features", outputCol = "scaledFeatures").fit(clean) 

In [ ]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = clean.randomSplit([0.7, 0.3])

In [ ]:
# Train a FM model
fm = FMClassifier(labelCol = "indexedLabel", featuresCol = "scaledFeatures", stepSize = 0.001)

In [ ]:
# Create a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureScaler, fm])

In [ ]:
# Train model
model = pipeline.fit(trainingData)

In [ ]:
# Make predictions
predictions2 = model.transform(testData)

In [ ]:
# Select example rows to display
predictions2.select("prediction", "indexedLabel", "features").show(5)

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       0.0|         0.0|(137,[0,1,2,3,4,5...|
|       0.0|         0.0|(137,[0,1,2,3,4,5...|
|       0.0|         0.0|(137,[0,1,2,3,4,5...|
|       0.0|         0.0|(137,[0,1,2,3,4,5...|
|       0.0|         0.0|(137,[0,1,2,3,4,5...|
+----------+------------+--------------------+
only showing top 5 rows



In [ ]:
# Select (prediction, true label) and compute test accuracy
evaluator = MulticlassClassificationEvaluator(
    labelCol = "indexedLabel", predictionCol = "prediction", metricName = "accuracy")
accuracy = evaluator.evaluate(predictions2)
print("Test set accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Test set accuracy = 0.82278
Test Error = 0.17722


In [ ]:
y_true = predictions2.select(['indexedLabel']).collect()
y_pred = predictions2.select(['prediction']).collect()

In [ ]:
print(confusion_matrix(y_true, y_pred))

[[49194     0]
 [10596     0]]


In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       0.82      1.00      0.90     49194
         1.0       0.00      0.00      0.00     10596

    accuracy                           0.82     59790
   macro avg       0.41      0.50      0.45     59790
weighted avg       0.68      0.82      0.74     59790



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
